In [1]:
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np

# --- Fechas ---
weeks = 52


In [2]:
import xmlrpc.client


# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')
df_merged["product_id"] = df_merged["product_id"].astype(str)

# Guardar
df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos
#para 105 semanas se demora aproximadamente 14 minutos

Facturas encontradas: 51566
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 80000 registros de líneas...
Descargados 100000 registros de líneas...
Descargados 120000 registros de líneas...
Descargados 140000 registros de líneas...
Descargados 160000 registros de líneas...
Descargados 180000 registros de líneas...
Descargados 200000 registros de líneas...
Descargados 201410 registros de líneas...
Total de líneas descargadas: 201410
Total de líneas descargadas con información de factura: 201410


In [3]:
df = df_merged.copy()

In [4]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["product_nom"] = df["product_name"].str.extract(r"\]\s*([A-Z]\w+)")


df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [5]:
df.head()

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
0,"[19349, '[BCS00505125] GS505 FILTRO COMBUSTIBL...",123200.0,1649727,2.0,[BCS00505125] GS505 FILTRO COMBUSTIBLE BALDWIN...,FMDE4411,2025-11-06 14:13:12,FMDE4411,447125,SUCURSAL MEDELLIN,WILSON PELAEZ,SUCURSALES / MEDELLÍN / EXTERNOS MED,36603,[CO9016151521] LUBRIRED S.A.S.,VENDEDOR EXTERNO,LUBRIRED S.A.S.,BD5,SUCURSAL MEDELLIN,BCS00505125,GS505,11,2025,6,BALDWIN
1,"[19518, '[BLS10023125] GS023A FILTRO ACEITE FI...",63400.0,1649726,2.0,[BLS10023125] GS023A FILTRO ACEITE FILTROFILTR...,FMDE4411,2025-11-06 14:13:12,FMDE4411,447125,SUCURSAL MEDELLIN,WILSON PELAEZ,SUCURSALES / MEDELLÍN / EXTERNOS MED,36603,[CO9016151521] LUBRIRED S.A.S.,VENDEDOR EXTERNO,LUBRIRED S.A.S.,BD5,SUCURSAL MEDELLIN,BLS10023125,GS023A,11,2025,6,BALDWIN
2,"[17268, '[DAB02946025] DA2946 FILTRO AIRE JCB,...",328800.0,1649725,6.0,"[DAB02946025] DA2946 FILTRO AIRE JCB,TEREX,M.F...",FCL61658,2025-11-06 14:08:01,FCL61658,447124,SUCURSAL CALLE 6,EDWIN FERNANDO COLLAZOS BARRETO,SUCURSALES / Ventas Credito Cll6,3114,[CO8300226123] J E FILTROS LTDA,VENDEDOR EXTERNO,J E FILTROS LTDA,BD2,SUCURSAL CALLE 6,DAB02946025,DA2946,11,2025,6,DONSSON
3,"[36798, '[DLS00213189] GS213 FILTRO ACEITE CHE...",6800.0,1649724,1.0,"[DLS00213189] GS213 FILTRO ACEITE CHEV, TOYOTA...",PV6E20043 Mostrador Barranquilla T3/3751,2025-11-06 13:57:40,PV6E20043,447123,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas mostrador B...,45060,[CC1047215478] BRANDON CARVAL CAMPO,MOSTRADOR,BRANDON CARVAL CAMPO,<NA>,SUCURSAL BARRANQUILLA,DLS00213189,GS213,11,2025,6,AUT*PARTS
4,"[17268, '[DAB02946025] DA2946 FILTRO AIRE JCB,...",54800.0,1649720,1.0,"[DAB02946025] DA2946 FILTRO AIRE JCB,TEREX,M.F...",FCAL6113,2025-11-06 13:49:21,FCAL6113,447122,SUCURSAL CALI,Administrador Administrador,Ecommerce,40205,[CO9016842887] ALMACEN TRACTO MACK DIESEL S.A....,VENDEDOR EXTERNO,ALMACEN TRACTO MACK DIESEL S.A.S. ZOMAC,BD9,SUCURSAL CALI,DAB02946025,DA2946,11,2025,6,DONSSON


## VENTAS PERDIDAS

In [6]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["fecha", "<=", FECHA_FIN], ["almacenamiento_tipo", "=", "agotado"]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

df_vp = pd.DataFrame(lost_sales_data)

# ===============================
# 4. Normalizar columnas
# ===============================

for col in ["store_id", "cliente_id"]:
    df_vp[col.replace('_id', '_name')] = df_vp[col].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else str(x) if x else None
    )
    df_vp[col + "_num"] = df_vp[col].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
    )

# Eliminar columnas originales
df_vp = df_vp.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# ===============================
# 4.1. Forzar tipos de columnas antes de exportar
# ===============================
for col in df_vp.columns:
    if col.endswith("_num"):
        df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").astype("Int64")
    else:
        df_vp[col] = df_vp[col].astype("string")

df_vp = df_vp.copy()



📅 Descargando registros desde 2024-11-07 hasta 2025-11-06


In [7]:
df_vp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69220 entries, 0 to 69219
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cantidad_reservada   69220 non-null  string
 1   product_ref          69220 non-null  string
 2   cantidad             69220 non-null  string
 3   almacenamiento_tipo  69220 non-null  string
 4   fecha                69220 non-null  string
 5   cantidad_existencia  69220 non-null  string
 6   origen               69220 non-null  string
 7   store_name           69220 non-null  string
 8   store_id_num         69220 non-null  Int64 
 9   cliente_name         69220 non-null  string
 10  cliente_id_num       69220 non-null  Int64 
dtypes: Int64(2), string(9)
memory usage: 5.9 MB


In [8]:
# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("M").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "mes", "ano", "Semana", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_month = lost_by_week


In [9]:
vp_month[vp_month["product_ref"]=="DAB08177025"]

,store_name,product_ref,mes,ano,Semana,dia,lost_sales,veces_vp
3786,PRINCIPAL COTA,DAB08177025,6,2025,2025-06-01,1,1.0,1
3787,PRINCIPAL COTA,DAB08177025,7,2025,2025-07-01,1,6.0,5
3788,PRINCIPAL COTA,DAB08177025,8,2025,2025-08-01,1,7.0,3
3789,PRINCIPAL COTA,DAB08177025,9,2025,2025-09-01,1,4.0,4
3790,PRINCIPAL COTA,DAB08177025,10,2025,2025-10-01,1,4.0,3
3791,PRINCIPAL COTA,DAB08177025,11,2025,2025-11-01,1,1.0,1
11408,SUCURSAL BARRANQUILLA,DAB08177025,9,2025,2025-09-01,1,6.0,1
11409,SUCURSAL BARRANQUILLA,DAB08177025,10,2025,2025-10-01,1,1.0,1
11410,SUCURSAL BARRANQUILLA,DAB08177025,11,2025,2025-11-01,1,1.0,1
17112,SUCURSAL BUCARAMANGA,DAB08177025,2,2025,2025-02-01,1,0.0,1


In [10]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")


df_vp.to_excel("/home/donsson/proyectos/VENTAS/prom_ventas/vp_mes.xlsx")

#Tiempo promedio 3 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20251106.xlsx


## VISUM

In [12]:
df.head()

,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,product_nom,mes,año,dia,marca
0,"[19349, '[BCS00505125] GS505 FILTRO COMBUSTIBL...",123200.0,1649727,2.0,[BCS00505125] GS505 FILTRO COMBUSTIBLE BALDWIN...,FMDE4411,2025-11-06 14:13:12,FMDE4411,447125,SUCURSAL MEDELLIN,WILSON PELAEZ,SUCURSALES / MEDELLÍN / EXTERNOS MED,36603,[CO9016151521] LUBRIRED S.A.S.,VENDEDOR EXTERNO,LUBRIRED S.A.S.,BD5,SUCURSAL MEDELLIN,BCS00505125,GS505,11,2025,6,BALDWIN
1,"[19518, '[BLS10023125] GS023A FILTRO ACEITE FI...",63400.0,1649726,2.0,[BLS10023125] GS023A FILTRO ACEITE FILTROFILTR...,FMDE4411,2025-11-06 14:13:12,FMDE4411,447125,SUCURSAL MEDELLIN,WILSON PELAEZ,SUCURSALES / MEDELLÍN / EXTERNOS MED,36603,[CO9016151521] LUBRIRED S.A.S.,VENDEDOR EXTERNO,LUBRIRED S.A.S.,BD5,SUCURSAL MEDELLIN,BLS10023125,GS023A,11,2025,6,BALDWIN
2,"[17268, '[DAB02946025] DA2946 FILTRO AIRE JCB,...",328800.0,1649725,6.0,"[DAB02946025] DA2946 FILTRO AIRE JCB,TEREX,M.F...",FCL61658,2025-11-06 14:08:01,FCL61658,447124,SUCURSAL CALLE 6,EDWIN FERNANDO COLLAZOS BARRETO,SUCURSALES / Ventas Credito Cll6,3114,[CO8300226123] J E FILTROS LTDA,VENDEDOR EXTERNO,J E FILTROS LTDA,BD2,SUCURSAL CALLE 6,DAB02946025,DA2946,11,2025,6,DONSSON
3,"[36798, '[DLS00213189] GS213 FILTRO ACEITE CHE...",6800.0,1649724,1.0,"[DLS00213189] GS213 FILTRO ACEITE CHEV, TOYOTA...",PV6E20043 Mostrador Barranquilla T3/3751,2025-11-06 13:57:40,PV6E20043,447123,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas mostrador B...,45060,[CC1047215478] BRANDON CARVAL CAMPO,MOSTRADOR,BRANDON CARVAL CAMPO,<NA>,SUCURSAL BARRANQUILLA,DLS00213189,GS213,11,2025,6,AUT*PARTS
4,"[17268, '[DAB02946025] DA2946 FILTRO AIRE JCB,...",54800.0,1649720,1.0,"[DAB02946025] DA2946 FILTRO AIRE JCB,TEREX,M.F...",FCAL6113,2025-11-06 13:49:21,FCAL6113,447122,SUCURSAL CALI,Administrador Administrador,Ecommerce,40205,[CO9016842887] ALMACEN TRACTO MACK DIESEL S.A....,VENDEDOR EXTERNO,ALMACEN TRACTO MACK DIESEL S.A.S. ZOMAC,BD9,SUCURSAL CALI,DAB02946025,DA2946,11,2025,6,DONSSON


In [50]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# -------- CONFIGURACIÓN --------
clientes = {
    "VISUM LTDA",
    "VILMAR LUBRICANTES S.A.S.",
    "ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.",
    "SODATRANS LTDA",
    "ESTACION Y LUBRICANTES CASABLANCA S.A.S"
}

# Asegurar que date_invoice es datetime
df["date_invoice"] = pd.to_datetime(df["date_invoice"])

# ✅ Usar price_subtotal como valor sin impuestos (Odoo ya es qty * price_unit)
df["amount_total"] = df["price_subtotal"]

# -------- DETECTAR ÚLTIMO MES CERRADO --------
hoy = datetime.today()

# Ej: si hoy es noviembre 2025, cerró octubre 2025
ultimo_mes_cerrado = pd.Period((hoy.replace(day=1) - relativedelta(days=1)), freq="M")
print("Último mes cerrado:", ultimo_mes_cerrado.strftime("%B %Y"))

# Últimos 6 meses previos al cerrado
meses_previos = pd.period_range(
    ultimo_mes_cerrado - 6,   # inicio (mes 7 hacia atrás)
    ultimo_mes_cerrado - 1,   # fin (6 meses antes del cerrado)
    freq="M"
)

# -------- FILTRAR DATOS --------
df_h = df[
    (df["client_name"].isin(clientes))
].copy()

# Crear columna año-mes
df_h["periodo"] = df_h["date_invoice"].dt.to_period("M")

# -------- AGRUPACIÓN MENSUAL --------
ventas_mensuales = (
    df_h.groupby(["client_name", "marca", "periodo"])["amount_total"]
    .sum()
    .reset_index()
)

# -------- CREAR BASE COMPLETA DE MESES PARA CADA CLIENTE + MARCA --------
base_completa = (
    pd.MultiIndex.from_product(
        [df_h["client_name"].unique(), df_h["marca"].unique(), meses_previos],
        names=["client_name", "marca", "periodo"]
    ).to_frame(index=False)
)

# Unimos: meses sin ventas quedan en 0
ventas_previos_6m = base_completa.merge(
    ventas_mensuales,
    on=["client_name", "marca", "periodo"],
    how="left"
).fillna({"amount_total": 0})

# -------- PROMEDIO 6 MESES --------
prom_6m = (
    ventas_previos_6m.groupby(["client_name", "marca"])["amount_total"]
    .mean()
    .reset_index()
    .rename(columns={"amount_total": "promedio_6_meses"})
)

# -------- VENTAS ÚLTIMO MES CERRADO --------
ventas_ultimo_mes = (
    ventas_mensuales[ventas_mensuales["periodo"] == ultimo_mes_cerrado]
    [["client_name", "marca", "amount_total"]]
    .rename(columns={"amount_total": "ventas_mes_cerrado"})
)

# -------- UNIR RESULTADOS --------
resultado = prom_6m.merge(
    ventas_ultimo_mes,
    on=["client_name", "marca"],
    how="left"
)

# -------- FORMATO FINAL --------
pd.set_option("display.float_format", "{:,.0f}".format)

resultado["promedio_6_meses"] = resultado["promedio_6_meses"].fillna(0).astype(int)
resultado["ventas_mes_cerrado"] = resultado["ventas_mes_cerrado"].fillna(0).astype(int)



print("Último mes cerrado detectado:", ultimo_mes_cerrado)


Último mes cerrado: October 2025
Último mes cerrado detectado: 2025-10


In [51]:
ventas_previos_6m.head(17)

,client_name,marca,periodo,amount_total
0,VISUM LTDA,BALDWIN,2025-04,"878,900"
1,VISUM LTDA,BALDWIN,2025-05,"22,803,600"
2,VISUM LTDA,BALDWIN,2025-06,"12,988,300"
3,VISUM LTDA,BALDWIN,2025-07,"235,800"
4,VISUM LTDA,BALDWIN,2025-08,"1,656,700"
5,VISUM LTDA,BALDWIN,2025-09,"19,954,600"
6,VISUM LTDA,DONSSON,2025-04,"260,100"
7,VISUM LTDA,DONSSON,2025-05,"17,056,400"
8,VISUM LTDA,DONSSON,2025-06,"11,304,000"
9,VISUM LTDA,DONSSON,2025-07,"662,800"


In [52]:
resultado.head(40)

,client_name,marca,promedio_6_meses,ventas_mes_cerrado
0,ESTACION Y LUBRICANTES CASABLANCA S.A.S,AUT*PARTS,0,0
1,ESTACION Y LUBRICANTES CASABLANCA S.A.S,BALDWIN,668633,2292500
2,ESTACION Y LUBRICANTES CASABLANCA S.A.S,DONSSON,99166,206800
3,ESTACION Y LUBRICANTES CASABLANCA S.A.S,OTRA,0,0
4,ESTACION Y LUBRICANTES CASABLANCA S.A.S,RACOR BRASIL,0,0
5,ESTACION Y LUBRICANTES CASABLANCA S.A.S,RACOR USA,0,0
6,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,AUT*PARTS,645683,0
7,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,BALDWIN,2708716,1658300
8,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,DONSSON,1185483,906700
9,ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.,OTRA,12266,0


In [54]:
resultado.to_excel("/home/donsson/proyectos/VENTAS/datasets_salida/prom_rutachiquinquira.xlsx")